# This is first part trying to predict a winner using Linear Regression

In [344]:
import numpy as np
import pandas as pd

## Data retrieval

### The following key is used to understand our data in details.
#### Key words
- ilkid : Player unique id
- firstname : name
- lastname: : surname
- leag: League that a player is playing for e.g. NBA or ABA
- gp: Games played
- minutes: Minute played
- pts: Points scored
- oreb: Offensive rebound
- dreb: Definsive rebound 
- reb: Rebounds
- asts: Assist to turnover ration
- stl: Steals
- blk: Blocks
- turnover: Losing ball to another team or throwing the ball out of bound.
- pf: Personal fouls
- fga: Field goal attampted
- fgm: Field goal made
- fta: Field goal 
- ftm: Free through made
- tpa: 3-Point Field Goals Attempted
- tpm: 3-Point Field Goals Made


'team', 'year', 'leag', 'o_fgm', 'o_fga', 'o_ftm', 'o_fta', 'o_oreb',
       'o_dreb', 'o_reb', 'o_asts', 'o_pf', 'o_stl', 'o_to', 'o_blk', 'o_3pm',
       'o_3pa', 'o_pts', 'd_fgm', 'd_fga', 'd_ftm', 'd_fta', 'd_oreb',
       'd_dreb', 'd_reb', 'd_asts', 'd_pf', 'd_stl', 'd_to', 'd_blk', 'd_3pm',
       'd_3pa', 'd_pts', 'pace', 'won', 'lost'
 
o_fgm and d_fgm (Field of goals made): This contains a teams goals that are scored in the field of play, usually contains the free throws, 2 points, and 3 points (Offensive and Defensive respectively.)

o_fga and d_fga (Field of goals attampted): This is just the field of goal attampted, both the successful and unsuccefull.

o_ftm and d_ftm (Free through made): This is just the 1 point goal made in the basket, usually awarded as result of what is called a techniqual foal in basketball.

o_fta and d_fta(Field of goals assist): This is just the pass that is madeand leads to a team member scoring

o_oreb and d_oreb (Offensive rebound and defensive) : a rebound, sometimes colloquially referred to as a board, is a statistic awarded to a player who retrieves the ball after a missed field goal or free throw

o_stl and d_stl (Offensive and defensive steals) : a steal occurs when a defensive player legally causes a turnover by their positive, aggressive action


In [345]:
# The following two dataframes contains the coaches information.
df_coaches_career = pd.read_csv('databasebasketball/coaches_career.txt')
df_coaches_season = pd.read_csv('databasebasketball/coaches_season.txt')

# The following contains indiidual player information and insite.
df_draft = pd.read_csv('databasebasketball/draft.txt') # --> Error {Performed some pre-processing}
df_player_allstar = pd.read_csv('databasebasketball/player_allstar.txt')
df_player_playoffs = pd.read_csv('databasebasketball/player_playoffs.txt')
df_player_playoffs_career = pd.read_csv('databasebasketball/player_playoffs_career.txt')
df_player_regular_season = pd.read_csv('databasebasketball/player_regular_season.txt')
df_player_regular_season_career = pd.read_csv('databasebasketball/player_regular_season_career.txt')
df_players = pd.read_csv('databasebasketball/players.txt')

# The following contains the teams data.
df_team_season = pd.read_csv('databasebasketball/team_season.txt')
df_teams = pd.read_csv('databasebasketball/teams.txt')

In [346]:
# df_player_allstar.head() 

# import collections
# all_star_appearances = collections.defaultdict(set)

# min_year = df_player_allstar['year'].min()
# max_year = df_player_allstar['year'].max()

# test_min = 1950
# test_max = 1952

# for year in range(min_year, max_year):
#     # for l_year in range(min_year, max_year):
#     all_stars_df = df_player_allstar.loc[df_player_allstar.year==year] 
#     name_surname = (all_stars_df['firstname']+' '+ all_stars_df['lastname']).tolist()
#     year = all_stars_df['year'].tolist()
    
#     max_elements = len(name_surname)
    
#     # add the elements in the map.
#     for index in range(0, max_elements):
#         all_star_appearances[name_surname[index]].add(year[index])
        
# # all_star_appearances

## Since certain matrix we available after 1970s our data start from that particular year

For example:
<b>Steals</b>, <b>blocks</b> and <b>turnovers</b> were not official nba stats until the 70's. Those 
stats are listed as zero for earlier season.

In [347]:
# The following two dataframes contains the coaches information.
# df_coaches_career = df_coaches_career.loc[df_coaches_career.year >= 1970]
df_coaches_season = df_coaches_season.loc[df_coaches_season.year >= 1970]

# # The following contains indiidual player information and insite.
# df_draft = df_draft.loc[df_draft.year >= '1970']
df_player_allstar = df_player_allstar.loc[df_player_allstar.year >= 1970]
df_player_playoffs = df_player_playoffs.loc[df_player_playoffs.year >= 1970] 
#df_player_playoffs_career = df_player_playoffs_career.loc[df_player_playoffs_career.year >= 1970]
df_player_regular_season = df_player_regular_season.loc[df_player_regular_season.year >= 1970]
#df_player_regular_season_career = df_player_regular_season_career.loc[df_player_regular_season_career.year >= 1970]
df_players = df_players.loc[df_players.firstseason >= 1970]

# # The following contains the teams data.
df_team_season = df_team_season.loc[df_team_season.year >= 1970]
# df_teams = df_teams.loc[df_teams.year >= 1970]

## The following attributes we deemed important for creating a feature vector

In [348]:
# len(df_player_regular_season.columns)
df_player_regular_season.head()

# drop certains columns in our data.
df = df_player_regular_season.drop(columns=['ilkid', 'year','firstname', 'lastname', 'team', 'leag', 'minutes', 'tpa', 'tpm'])
df.dtypes

df_team_season.year.max()

2004

## Determing a winning team
<i>For this implementation a linear regression model was used: </i>

    * We will calculate a win to loss ratio and make that as our target value.
        * Calculated as follows:
              * win_to_loss_ratio = (win / win + loss)

In [349]:
# Perform some data preprocessing

season_team_stat = df_team_season.drop(columns=['team', 'year', 'leag', 'pace'])
teams = df_team_season['team']

import math

def win_to_loss(won, loss):
    return math.floor((won / (won + loss)) * 100)

# added the target column in season_team_stat
season_team_stat['win_loss_percent'] = season_team_stat.apply(lambda x: win_to_loss(x.won, x.lost), axis=1)

In [350]:
from sklearn import linear_model

# create a linear regression object.
reg = linear_model.LinearRegression()

# input values.
X_input = season_team_stat.drop(columns=['won', 'lost', 'win_loss_percent'])
Y_input = season_team_stat.win_loss_percent

# fit our model.
reg.fit(X_input, Y_input)

LinearRegression()

In [351]:
# Let us look at the coeffience of our model
reg.coef_

array([-1.81982791e-01, -2.50692817e-04, -9.17571855e-02,  5.10110362e-04,
        2.23839045e-03,  3.12538103e-03, -2.13574822e-03,  1.03818978e-03,
       -3.66203820e-03,  5.29162338e-03, -1.82197098e-03,  2.36635999e-03,
       -8.65176007e-02, -6.91378765e-04,  1.26073997e-01, -6.32086824e-01,
        5.49391224e-03, -3.20829173e-01,  6.19984023e-03,  9.32646499e-04,
       -2.62519252e-03, -2.02820330e-03, -1.53541888e-03,  4.03530523e-03,
       -5.47918926e-03,  6.21048468e-04, -8.97843185e-03, -3.16575878e-01,
        4.85875282e-04,  2.78317438e-01])

In [352]:
# Let us look at the way intercept.
reg.intercept_

43.53436867092078

## Let us try testing the model

In [353]:
import random as rd

# guess1 = rd. randint(0,len(teams))
# guess2 = rd. randint(0,len(teams
                           
guess1 = 272
guess2 = 500

team1 = season_team_stat.iloc[guess1].drop(['won','lost','win_loss_percent'])
team2 = season_team_stat.iloc[guess2].drop(['won','lost','win_loss_percent'])

In [354]:
pred_team1 = reg.predict([team1])
pred_team1

array([44.03705042])

In [355]:
pred_team2 = reg.predict([team2])

In [356]:
if pred_team1 > pred_team2:
    print(f'The following team is predicted to win: {teams[guess1]}')
else:
    print(f'The following team is predicted to win: {teams[guess1]}')

The following team is predicted to win: ATL


## Testing the error of our model, since we using a regression model

In [357]:
# Calculate the mean-squered error.

# example of calculate the mean squared error
from sklearn.metrics import mean_squared_error

y_true = season_team_stat.win_loss_percent
y_pred = reg.predict(X_input)

# calculate mean squared error
errors = mean_squared_error(y_true, y_pred)

print(f'The mean squared error of our model is: {errors}%')

The mean squared error of our model is: 15.518814431580513%


In [358]:
# calculate root mean square error
errors = mean_squared_error(y_true, y_pred, squared=False)

print(f'The root mean squared error of our model is: {errors}%')

The root mean squared error of our model is: 3.9393926475512075%


# Second method using SVM